<a href="https://colab.research.google.com/github/har1zxje/detect_fisheye8k/blob/main/fisheye8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kết nối Google Drive
drive.mount('/content/drive')

!pip install ultralytics

In [ ]:
import textwrap

# --- Tạo file data.yaml ---
# Đường dẫn để lưu file YAML
data_yaml_path = '/content/drive/MyDrive/fisheye8k_project/data.yaml'

# Gộp nội dung vào một biến và dùng textwrap.dedent để tránh lỗi thụt lề
yaml_content = textwrap.dedent("""
    train: /content/drive/MyDrive/fisheye8k_project/Fisheye8K_data/train/images
    val: /content/drive/MyDrive/fisheye8k_project/Fisheye8K_data/test/images
    nc: 5
    names: ["Bus", "Bike", "Car", "Pedestrian", "Truck"]
""")

# Chỉ cần một lần ghi file duy nhất
with open(data_yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Đã tạo file {data_yaml_path} thành công.")

In [ ]:
from ultralytics import YOLO

# Nâng cấp lên model YOLOv10-L
model = YOLO('yolov10l.pt')
data_yaml_path_on_drive = '/content/drive/MyDrive/fisheye8k_project/data.yaml'

# Bắt đầu huấn luyện với model lớn hơn và augmentation được tinh chỉnh
print("🚀 Bắt đầu huấn luyện MỚI với model YOLOv10-L và augmentation nâng cao...")
results = model.train(
    data=data_yaml_path_on_drive,
    epochs=50,
    imgsz=640,
    batch=8,
    project='/content/drive/MyDrive/fisheye8k_project/training_results',
    name='fisheye_run_yolov10l_enhanced_aug',

    # Các tham số augmentation hình học (giữ nguyên)
    degrees=10.0,
    translate=0.1,
    scale=0.7,
    shear=0.2,
    perspective=0.001,
    flipud=0.5,
    fliplr=0.5,

    # Các tham số augmentation phức hợp (giữ nguyên)
    mosaic=1.0,
    mixup=0.1,

    # --- THAM SỐ MỚI ---
    # Điều chỉnh không gian màu HSV
    hsv_h=0.015,  # Thay đổi tông màu
    hsv_s=0.7,    # Thay đổi độ bão hòa
    hsv_v=0.4,    # Thay đổi độ sáng

    # Bật copy_paste
    copy_paste=0.1 # Xác suất áp dụng là 10%
)

In [ ]:
from ultralytics import YOLO

# --- TRỎ ĐẾN FILE CHECKPOINT CŨ ---
# Đây là tệp chứa "bộ não" của model mà bạn muốn tiếp tục huấn luyện
path_to_best_checkpoint = '/content/drive/MyDrive/fisheye8k_project/training_results/fisheye_run_yolov10l_enhanced_aug/weights/best.pt'

# --- TẢI MODEL VÀ TRỌNG SỐ TỪ CHECKPOINT ---
# Thao tác này sẽ tải cả kiến trúc và trọng số đã được huấn luyện từ file cũ
model = YOLO(path_to_last_checkpoint)

# --- BẮT ĐẦU MỘT LẦN CHẠY MỚI ---
print("🚀 Bắt đầu một lần huấn luyện MỚI, kế thừa trọng số từ lần chạy trước...")
results = model.train(

    data='/content/drive/MyDrive/fisheye8k_project/data.yaml',
    epochs=200,
    batch=8,
    project='/content/drive/MyDrive/fisheye8k_project/training_results',
    name='fisheye_run_yolov10l_enhanced_aug_continued',

    compile=True,
    patience=50
)

print("🎉 Quá trình huấn luyện đã hoàn tất!")

In [ ]:
from ultralytics import YOLO

# --- ĐƯỜNG DẪN ĐẾN MODEL TỐT NHẤT ---
path_to_best_checkpoint = '/content/drive/MyDrive/fisheye8k_project/training_results/fisheye_run_yolov10l_enhanced_aug_continued2/weights/best.pt'

# --- TẢI MODEL MỚI VỚI TRỌNG SỐ TỪ 'best.pt' ---
# Thao tác này sẽ tạo một model mới và nạp "kiến thức" từ file best.pt vào
model = YOLO(path_to_best_checkpoint)

# --- BẮT ĐẦU MỘT LẦN HUẤN LUYỆN MỚI ---
print(f"🚀 Bắt đầu huấn luyện model MỚI, kế thừa trọng số từ: {path_to_best_checkpoint}")
results = model.train(

    data='/content/drive/MyDrive/fisheye8k_project/data.yaml',
    epochs=500,
    imgsz=1280,
    batch=4,
    project='/content/drive/MyDrive/fisheye8k_project/training_results',

    name='yolov10l_final_1',

    compile=True,
    patience=50,

    # --- THAM SỐ MỚI: KIỂM SOÁT LEARNING RATE ---
    lr0=0.001,
    lrf=0.005,
    weight_decay=0.0005,
    patience=20,

    # Các tham số augmentation
    degrees=10.0,
    translate=0.1,
    shear=0.2,
    perspective=0.001,
    mosaic=1.0,
    mixup=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    scale=0.9,

    save_period=5,
    exist_ok=True,
)

print("\n🎉 Quá trình huấn luyện mới đã hoàn tất!")